# Initialization

In [1]:
import pandas as pd
import seaborn as sns
from datasets import Dataset
from datasets import load_metric
import jiwer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm # tqdm is used to show progress bar
import re # re is used for regular expressions
import os # os is used for operating system related functions
import torch # torch is used for building deep learning models
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

RuntimeError: Failed to import transformers.data.data_collator because of the following error (look up to see its traceback):
Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [16]:
import pandas as pd
df = pd.read_csv("submission_jup_eightt_two.csv")
dff = pd.read_csv('testData.csv')

In [18]:
df.tail(50)

,row_id_column_name,ipa
27178,27178,ɐʃɐmid̪eɾ ʃɔbɐ͡i̯ ɟɔgonnɐt̪ʰ biʃʃobid̪d̪ɐlɔʲ ʃ...
27179,27179,"t̪ɐɾ kɔt̪ʰɐʲ, ‘biʃbɐɾ kʰelle ɛkbɐɾ t̪o hɐɾt̪e ..."
27180,27180,ɐmɐɾ biʃʃɐʃ d̪ɔɾʃokd̪eɾ moneɾ mɔt̪o ekti cɔloc...
27181,27181,ɔnnot̪ʰɐʲ kʰɐd̪d̪e ʃʲɔŋʃɔmpuɾno t̪ʰɐkɐɾ ʃɔpno ...
27182,27182,tɐnɐ bɾiʃti ɐɾ pɐhɐɽi dʱɔle e͡i̯ ʃɔɽokeɾ od̪ʱi...
27183,27183,"t̪ɔbe st̪ʰɐniʲoɾɐ ɟɐnɐn, ʃiʃuti ɐd̪o͡u̯ guli c..."
27184,27184,e gʱɔtonɐʲ ɾɐt̪e͡i̯ puliʃ obʱiɟɐn cɐliʲe kiɾic...
27185,27185,t̪ini bɔŋgobond̪ʱu ɟɐd̪ugʱɔɾ poɾid̪ɔɾʃon kɔɾen...
27186,27186,"d̪okɐneɾ bɛŋk ɛkɐ͡u̯nt nɔmbɔɾ d̪in, ɐmi tɐkɐ d..."
27187,27187,ʃe͡i̯ mɐmolɐɾ t̪ɔd̪ont̪o koɾcʰe puliʃ।


In [17]:
dff.head(50)

,row_id_column_name,text
0,0,বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশ...
1,1,এ নিয়ে বিবাদে ২০১৫ সালের ২ জুন রাত সাড়ে ১১টায় ...
2,2,আজ থেকে ১৪ বছর আগে তিনি চলে গেছেন না ফেরার দেশে।
3,3,নিহত ব্যক্তি কুতপালং টালের ই-২ ব্লকের আবুল বাছ...
4,4,সংক্ষিপ্ত স্কোরশ্রীলংকা প্রথম ইনিংস ৪৮২ (করুনা...
5,5,"এগুলোর মধ্যে সাজ্জাদ হোসেনের ‘নন স্টপ’, আশুতোষ..."
6,6,জব্দ মাদকসহ ট্রলারের মূল্য আট লাখ ৫২ হাজার টাক...
7,7,মা আয়েশা বেগম (৫০) গৃহিণী।
8,8,কক্সবাজার : জেলার উখিয়ায় যাত্রীবাহী বাসের ধাক্...
9,9,"মানববন্ধনে বক্তারা বলেন, ছাত্রলীগ সাংবাদিকদের ..."


# Data pre-processing

In [2]:
train_df = pd.read_csv("trainIPAdb.csv")
train_df = train_df.drop(columns = ['Unnamed: 0'])

In [3]:
alpha_pat = "[a-zA-z0-9]"

train_df["text"] = train_df["text"].str.replace(alpha_pat, "", regex=True)

In [4]:
train_df, val_df = train_test_split(train_df, test_size=0.1, shuffle=True, random_state=3000)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

# Dataset

In [5]:
ds_train = Dataset.from_pandas(train_df)
ds_eval = Dataset.from_pandas(val_df)

# Model

In [6]:
model_id = "google/umt5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [7]:
def prepare_dataset(sample):
    output = tokenizer(sample["text"])
    output["labels"] = tokenizer(sample["ipa"])['input_ids']
    output["length"] = len(output["labels"])
    return output


ds_train = ds_train.map(prepare_dataset, remove_columns=ds_train.column_names)
ds_eval = ds_eval.map(prepare_dataset, remove_columns=ds_eval.column_names)

Map:   0%|          | 0/19799 [00:00<?, ? examples/s]

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

# Metric

In [8]:
wer_metric = load_metric("wer")


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = wer_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"wer": result}

C:\Users\Niaz Mohaiman Abtahi\AppData\Local\Temp\ipykernel_36352\3914945023.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


# Training

In [9]:
model_id = "iit-eight"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_id,
    group_by_length=True,
    length_column_name="length",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    metric_for_best_model="wer",
    greater_is_better=False,
    load_best_model_at_end=True,
    num_train_epochs=15,
    save_steps=2000,
    eval_steps=2000,
    logging_steps=2000,
    learning_rate=5e-4,
    weight_decay=1e-2,
    warmup_steps=1000,
    save_total_limit=8,
    predict_with_generate=True,
    generation_max_length=128,
    push_to_hub=False,
    report_to="none",
)

In [10]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Wer
2000,2.487800,0.106440,0.116600
4000,0.145800,0.055140,0.058385
6000,0.085700,0.046948,0.048633
8000,0.060500,0.037760,0.037057
10000,0.047600,0.037173,0.035446
12000,0.035400,0.036107,0.035065
14000,0.028100,0.033444,0.029637
16000,0.023200,0.032083,0.026076
18000,0.019700,0.030589,0.025737
20000,0.015900,0.031783,0.026712


TrainOutput(global_step=37125, training_loss=0.16179889827946622, metrics={'train_runtime': 141559.6102, 'train_samples_per_second': 2.098, 'train_steps_per_second': 0.262, 'total_flos': 2.292348211506893e+16, 'train_loss': 0.16179889827946622, 'epoch': 15.0})

In [11]:
trainer.save_model(model_id)